Задание:
Вывести список групп в ВК в которых состоит пользователь, но не состоит никто из его друзей.

Входные данные:
Имя пользователя или его id в ВК, для которого мы проводим исследование.

Требования к программе:
Программа не падает, если один из друзей пользователя помечен как “удалён” или “заблокирован”.
Показывает что не зависла: рисует точку или чёрточку на каждое обращение к api.
Не падает, если было слишком много обращений к API (Too many requests per second). Ограничение от ВК: не более 3х обращений к API в секунду. Могут помочь модуль time (time.sleep) и конструкция (try/except).
Код программы удовлетворяет PEP8.
Не использовать внешние библиотеки (vk, vkapi).

In [ ]:
import requests
#import time
import json

access_token = '958eb5d439726565e9333aa30e50e0f937ee432e927f0dbd541c541887d919a7c56f95c04217915c32008'
VERSION = 5.107


class User:
    def __init__(self, u_id):
        self.id = u_id

        url = 'https://api.vk.com/method/users.get'
        params = {'access_token': access_token,
                  'user_ids': u_id,
                  'v': VERSION
                  }

        data = self.get_response(url, params)

        if data.get('response'):
            self.id = data['response'][0]['id']
        elif data.get('error'):
            self.id = data['error']['error_msg']


    # request -> response
    def get_response(self, url, params):
        response = requests.get(url, params=params)

        if response.status_code == 200:
            data = response.json()

        return data


    # Список групп пользователя
    def get_groups(self):
        
        url = f'https://api.vk.com/method/groups.get'
        params = {'access_token': access_token,
                  'user_id': self.id,
                  'extended': 1,
                  'fields': 'members_count,counters,contacts,city,country',
                  'count': 1000,
                  'v': VERSION
                  }

        data = self.get_response(url, params)

        if data.get('error'):
            return 'Ошибка id'
        else:
            groups = data['response']['items']
            return groups


    # Количество друзей пользователя в группе
    def get_group_contacts_count(self, g_id):
        url = 'https://api.vk.com/method/groups.getMembers'
        params = {'access_token': access_token,
                  'group_id': g_id,
                  'filter': 'friends',
                  'v': VERSION
                  }

        data = self.get_response(url, params)

        if data.get('response'):
            return data['response']['count']


    # Список уникальных групп
    def get_uniq_groups(self):
        if self.get_groups() != 'Ошибка id':
            groups_list = self.get_groups()
            uniqal_groups = []
            print(f'Всего групп у пользователя: {len(groups_list)}')

            for group in groups_list:
                print('.')
                #time.sleep(1)
                count_of_friends = self.get_group_contacts_count(group['id'])

                if count_of_friends == 0:
                    g_data = {'name': group['name'],
                              'gid': group['id'],
                              'members_count': group['members_count']
                              }
                    uniqal_groups.append(g_data)

            if len(uniqal_groups) == 0:
                print('Done')
                print('Не состоит ни в одной уникальной группе.')
            else:
                with open('groups.json', 'w', encoding='utf8') as f:
                    json.dump(uniqal_groups, f, ensure_ascii=False)

                print('Done')
                print(f'Найдено уникальных групп: {len(uniqal_groups)}')
                print('Список уникальных групп в файле "groups.json"')
        else:
            print('Ошибка id')


def get_profile():
    user_input = input('Введите id пользователя: ')
    return User(user_input)


get_profile().get_uniq_groups()